In [40]:
suppressPackageStartupMessages({
library(dplyr)
library(Seurat)
library(readr)
library(ggplot2)
library(tidyr)
library(ggrepel)
library(viridis)
library(tibble)
library(data.table)
library(pheatmap)
library(patchwork)
library(stringr)
})

In [41]:
set.seed(1234)

In [3]:
setwd("/staging/leuven/stg_00041/Ryan/Irene_NCB")

In [4]:
data <- readRDS("/lustre1/project/stg_00041/Irene/10X_NAIVE_TSC_2021/Integration_plots/Pet_Zhou_IT_allgenes_norm.rds")

In [5]:
data_IT <- readRDS("/staging/leuven/stg_00041/Irene/10X_NAIVE_TSC_2021/nonaggr_noMEFs_mouse_human_analysis/seurat_object_norm.rds")

In [6]:
data_IT@meta.data$cluster_PRC2i <- NA
data_IT@meta.data[data_IT@meta.data$seurat_clusters==0,]$cluster_PRC2i <- "Naive"
data_IT@meta.data[data_IT@meta.data$seurat_clusters==1,]$cluster_PRC2i <- "Intermediate"
data_IT@meta.data[data_IT@meta.data$seurat_clusters==2,]$cluster_PRC2i <- "TSCs"
head(data_IT@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,sample,RNA_snn_res.0.2,seurat_clusters,cluster_PRC2i
,<chr>,<dbl>,<int>,<dbl>,<chr>,<fct>,<fct>,<chr>
TSC_AAACCCACACTCCGGA.1,naive_TSC_PRC2i_Irene_VP,79984,8416,12.40998,TSC,1,1,Intermediate
TSC_AAACGAAGTAAGGCCA.1,naive_TSC_PRC2i_Irene_VP,187905,10316,13.33493,TSC,1,1,Intermediate
TSC_AAACGCTTCTACTGAG.1,naive_TSC_PRC2i_Irene_VP,103178,8633,12.52302,TSC,2,2,TSCs
TSC_AAAGGGCCATGGAATA.1,naive_TSC_PRC2i_Irene_VP,70210,8112,18.62555,TSC,1,1,Intermediate
TSC_AAAGGTAAGATGCTGG.1,naive_TSC_PRC2i_Irene_VP,67006,7910,15.51354,TSC,1,1,Intermediate
TSC_AAAGTCCAGAAACACT.1,naive_TSC_PRC2i_Irene_VP,56056,7707,16.82068,TSC,1,1,Intermediate


In [7]:
tail(data_IT@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,percent.MT,sample,RNA_snn_res.0.2,seurat_clusters,cluster_PRC2i
,<chr>,<dbl>,<int>,<dbl>,<chr>,<fct>,<fct>,<chr>
NAIVEPRC2i_TTTGGAGGTACATACC.1,naive_TSC_PRC2i_Irene_VP,18009,4697,7.357432,NAIVEPRC2i,0,0,Naive
NAIVEPRC2i_TTTGGAGGTCACTCTC.1,naive_TSC_PRC2i_Irene_VP,8016,2946,10.491517,NAIVEPRC2i,0,0,Naive
NAIVEPRC2i_TTTGGTTGTATGAGCG.1,naive_TSC_PRC2i_Irene_VP,11078,3187,9.351869,NAIVEPRC2i,0,0,Naive
NAIVEPRC2i_TTTGGTTTCCCAGCGA.1,naive_TSC_PRC2i_Irene_VP,9382,3280,8.878704,NAIVEPRC2i,0,0,Naive
NAIVEPRC2i_TTTGTTGCAGACGCTC.1,naive_TSC_PRC2i_Irene_VP,14288,4317,7.691769,NAIVEPRC2i,0,0,Naive
NAIVEPRC2i_TTTGTTGCATCGGAAG.1,naive_TSC_PRC2i_Irene_VP,16422,4497,15.077335,NAIVEPRC2i,0,0,Naive


In [8]:
for(i in rownames(data_IT@meta.data)){
    if(grepl("PRC2i", i)==TRUE){
        data_IT@meta.data[i,"cluster_PRC2i"] <- paste(data_IT@meta.data[i,"cluster_PRC2i"], "_PRC2i")
        }
    }

In [9]:
signature_markers_PRG<- list()
signature_markers_PRG[["Trophoblast"]] <- c("KRT18", "TFAP2C", "GATA2", "GATA3", "KRT7", "CLDN4", "HAND1", "ITGA2", "VGLL1", "TP63", "HLA-G")
signature_markers_PRG[["Core"]] <- c( "NANOG", "POU5F1", "SOX2", "SUSD2")
#15
signature_markers_PRG[["Naive"]] <- c( "VENTX", "DNMT3L", "KLF4", "SPIC", "FGF4", "KLF17", "TFCP2L1")
signature_markers_PRG[["Primed"]] <- c("ZIC2", "OTX2", "DUSP6")
#25
signature_markers_PRG[["PrE"]] <- c("SOX17", "GATA4", "GATA6", "FOXA2", "PDGFRA")
#30
signature_markers_PRG[["Mesoderm"]] <- c("EPHA4", "GSC", "HOPX", "CDX1", "CDX2", "CDX4", "HAND2", "FOXF1", "HEY1", "MIXL1", "MESP1", "ZIC3")
#42
signature_markers_PRG[["Endoderm"]] <- c("CER1", "CXCR4", "FOXA1", "HNF1B", "HHEX", "KIT", "EOMES")
signature_markers_PRG[["Ectoderm"]] <- c("COL2A1", "DRD4", "EN1", "LMX1A", "NR2F1", "PAPLN", "POU4F1", "PAX3", "PAX6", "SOX1", "WNT1", "ZBTB16")

In [10]:
data_meta.data_bk <- data@meta.data

In [11]:
data@meta.data[rownames(data_IT@meta.data[which(grepl("TSC", rownames(data_IT@meta.data))),]), "finalClusters"] <- data_IT@meta.data[which(grepl("TSC", rownames(data_IT@meta.data))), "cluster_PRC2i"]

In [14]:
data@meta.data$finalClusters %>% unique()

[1] "TB.medium2"          NA                    "TB.late"            
 [4] "Pre.ST"              "TB.medium3"          "PrE"                
 [7] "Pre.EVT"             "EPI"                 "ST"                 
[10] "EVT"                 "ysTE"                "early_TE"           
[13] "medium_TE"           "TB.medium1"          "TB.early"           
[16] "B1_B2"               "TB.apoptosis"        "late_TE"            
[19] "pre_implantation"    "Intermediate"        "TSCs"               
[22] "Naive"               "TSCs _PRC2i"         "Intermediate _PRC2i"

In [12]:
data@meta.data$finalClusters <- factor(data@meta.data$finalClusters, levels=rev(c('TSCs', 'TSCs _PRC2i', 'Intermediate', 'Intermediate _PRC2i', 'Naive', 'EPI', 'TB.early', 'TB.medium1', 'TB.medium2', 'TB.medium3', 'TB.late', 'TB.apoptosis', 'PrE', 'Pre.ST', 'ST', 'Pre.EVT', 'EVT', 'early_TE', 'medium_TE', 'late_TE', 'ysTE', 'Unknown', 'pre_implantation')))

In [16]:
data_filt_dotplot_full <- data %>% subset(cells = rownames(data@meta.data[which(data@meta.data$finalClusters %in% c("TSCs", "TSCs _PRC2i", "Intermediate", "Intermediate _PRC2i", "EPI", "TB.early", "PrE")),]))

In [25]:
options(repr.plot.width = 11.5, repr.plot.height = 2.5, repr.plot.res = 150)
pdf("dotplot_IPZ_d4only_nonaivecluster_select_annotations_noNR2F2_size_corrected.pdf", width = 11.5, height = 2.5)
DotPlot(data_filt_dotplot_full, features = signature_markers_PRG, assay="RNA", 
               dot.scale = 3, group.by="finalClusters")+
    RotatedAxis()+
scale_colour_gradient2(low = "grey90", mid = "tomato", high = "tomato4") & theme(text = element_text(size=7), axis.text = element_text(size=7), axis.text.x = element_text(angle=90, hjust=1, vjust=0.5), legend.box = "horizontal")
dev.off()

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.



png 
  2

In [26]:
data_simple <- readRDS("/lustre1/project/stg_00041/Irene/10X_NAIVE_TSC_2021/nonaggr_noMEFs_mouse_human_analysis/Pet_Zhou_IT_allgenes_norm_new_anno.rds")

In [27]:
data_simple@meta.data$finalClustersMerged %>% unique()

[1] "Embryo Trophoblast" "unknown"            "Embryo ST"         
 [4] "PrE"                "Embryo EVT"         "Epiblast"          
 [7] "ysTE"               "Embryo TE"          "(Pre) Blastocyst"  
[10] "EightCells"         "Intermediate"       "TSCs"              
[13] "Naive"              "TSCs_PRC2i"         "Intermediate_PRC2i"

In [33]:
data_filt_dotplot_simple <- data_simple %>% subset(cells = rownames(data_simple@meta.data[which(data_simple@meta.data$finalClustersMerged %in% c("TSCs", "TSCs_PRC2i", "Intermediate", "Intermediate_PRC2i", "Epiblast", "Embryo Trophoblast", "PrE")),]))

In [34]:
data_filt_dotplot_simple@meta.data$finalClustersMerged <- factor(data_filt_dotplot_simple@meta.data$finalClustersMerged, levels=rev(c("TSCs", "TSCs_PRC2i", "Intermediate", "Intermediate_PRC2i", "Epiblast", "Embryo Trophoblast", "PrE")))

In [35]:
data_filt_dotplot_simple@meta.data$finalClustersMerged %>% unique()

[1] Embryo Trophoblast PrE                Epiblast           Intermediate      
[5] TSCs               TSCs_PRC2i         Intermediate_PRC2i
7 Levels: PrE Embryo Trophoblast Epiblast Intermediate_PRC2i ... TSCs

In [39]:
options(repr.plot.width = 11.5, repr.plot.height = 2.5, repr.plot.res = 150)
pdf("dotplot_IPZ_d4only_nonaivecluster_select_annotations_noNR2F2_size_corrected_all_trophoblast.pdf", width = 11.5, height = 2.5)
DotPlot(data_filt_dotplot_simple, features = signature_markers_PRG, assay="RNA", 
               dot.scale = 3, group.by="finalClustersMerged")+
    RotatedAxis()+
scale_colour_gradient2(low = "grey90", mid = "tomato", high = "tomato4") & 
theme(text = element_text(size=7), axis.text = element_text(size=7), axis.text.x = element_text(angle=90, hjust=1, vjust=0.5), legend.box = "horizontal") &
scale_y_discrete(name="", labels=rev(c("Trophoblast -PRC2i", "Trophoblast +PRC2i", "Intermediate -PRC2i", "Intermediate +PRC2i", "Epiblast", "Trophoblast", "Primitive endoderm")))
dev.off()

Scale for 'colour' is already present. Adding another scale for 'colour',
which will replace the existing scale.



png 
  2